In [ ]:
import numpy as np
import pandas as pd
import glob
import time
import pandas as pd
import os
import matplotlib.pyplot as plt
import keras

import cv2

In [ ]:
df=pd.read_csv('/kaggle/input/cross-site-scripting-xss-dataset-for-deep-learning/XSS_dataset.csv', encoding='utf-8-sig')

In [ ]:
df.head()

In [ ]:
df=df[df.columns[-2:]]  #get sentence and labels

In [ ]:
df.head()

In [ ]:
#sentence data from data frame
sentences=df['Sentence'].values
sentences[1]

In [ ]:
print(len(sentences))

In [ ]:
def convert_to_ascii(sentence):
    sentence_ascii=[]

    for i in sentence:
        
            if (ord(i)<=128): #asccii less than 128 discard it
                    sentence_ascii.append(ord(i))
    
            else:
                    pass
            

    pad=np.zeros((10000))

    for i in range(len(sentence_ascii)):
        zer[i]=sentence_ascii[i]

    pad.shape=(100, 100) #shape for input layer

    return pad


In [ ]:
arr=np.zeros((len(sentences),100,100))

for i in range(len(sentences)):
    
    feature=convert_to_ascii(sentences[i]) #convert each sentence into ascii and the appropriate batch size

    x=np.asarray(feature,dtype='float')
    feature =  cv2.resize(x, dsize=(100,100), interpolation=cv2.INTER_CUBIC)
    feature/=128    
    arr[i]=feature



In [ ]:
#shape data to be input to CNN
data = arr.reshape(arr.shape[0], 100, 100, 1) #(100,100,1)

In [ ]:
y=df['Label'].values
df.head()

In [ ]:
from sklearn.model_selection import train_test_split
indices = np.arange(data.shape[0])
trainX, testX, trainY, testY, train_index, testX_index = train_test_split(
    data, y, indices, test_size=0.2, random_state=42
) #split into testing and trainng set

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout,MaxPool2D, BatchNormalization

In [ ]:
# A basic CNN Model
# Number of layers = 11
# Number of Convolutional layer: 3

model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64,(3,3), activation=tf.nn.relu, input_shape=(100,100,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

In [ ]:
model.compile(loss='binary_crossentropy',  #loss calcuated with cross entropy
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

In [ ]:
#validation accuracy > 97

class acc(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.97):
            print(" \n Reached 97% + validation accuracy")
            self.model.stop_training=True
back = acc()

In [ ]:
batch_size = 128
num_epoch = 20
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY))

In [ ]:
model.save('xss_detection_model.h5')

In [ ]:
model = tf.keras.models.load_model('/kaggle/input/model-h5-cnn/xss_detection_model.h5') #laod model

In [ ]:
#predit on test set
pred=model.predict(testX)


In [ ]:
#thoreshold predicted values

for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [ ]:
true=0
false=0

for i in range(len(pred)):
    if pred[i] == testY[i]:
        true+=1
    else:
        false+=1
        print("Wrong predictions--> ",sentences[testX_index[i]])
        
print("correct predicted :: ", true)
print("false prediction :: ", false)

In [ ]:
# Number of attack and benign data in test set 

attack=0
benign=0
for i in range(len(testY)):
    if testY[i]==1:
        attack+=1
        print("XSS ->", sentences[testX_index[i]])
    else:
        benign+=1
        print("not XSS ->", sentences[testX_index[i]])


print("Attack data in test set :: ", attack)
print(" Benign data in test set :: ", benign)

In [ ]:
def predict_xss(sentence, model):
    ascii_sentence = convert_to_ascii(sentence)
        x = np.asarray(ascii_sentence, dtype='float')
    processed_input = cv2.resize(x, dsize=(100, 100), interpolation=cv2.INTER_CUBIC)
    processed_input /= 128 
    processed_input = processed_input.reshape(1, 100, 100, 1)
    
    #prediction
    prediction = model.predict(processed_input)

    is_xss = prediction[0][0] > 0.5
    return "XSS" if is_xss else "Not XSS"


input_sentence = "<IMG SRC=javascript:alert(&quot;XSS&quot;)>" #this would pass HTML encoding
result = predict_xss(input_sentence, model)
print(result, "-->", input_sentence)

### Confusion Matrix

In [ ]:
#true positive, true negative, false positive and false negative to get accuracy, precison and recall

def accuracy(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

def precision(tp,fp): #true postivie and false positive
    
    precision = tp / (tp+fp)
    
    return precision

def recall(tp,fn): #true positive and false negative
    
    recall=tp / (tp+fn)
    
    return recall

In [ ]:
def confusion_matrix(true,pred):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(true,pred):
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy(true_positive, true_negative, false_positive, false_negative)
    precision=precision(true_positive, false_positive)
    recall=recall(true_positive, false_negative)
    
    matrix = np.array([[true_negative, false_positive],
                   [false_negative, true_positive]])

    return (accuracy,precision,recall,matrix)

In [ ]:
accuracy,precision,recall,cm=confusion_matrix(testY,pred)

In [ ]:
print(" Accuracy : {0} \n Precision : {1} \n Recall : {2}".format(accuracy, precision, recall))

In [ ]:
import seaborn as sns  
import matplotlib.pyplot as plt  


#dislay matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()